In [1]:
import numpy as np
import pandas as pd
import os
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

In [2]:
ROOT_TRAIN = 'data/new_train/'
ROOT_TEST = 'data/new_test/'

train_paths = os.listdir(ROOT_TRAIN)
test_paths = os.listdir(ROOT_TEST)

In [3]:
train_dfs = []
test_dfs = []

for key, path in enumerate(train_paths):
    train_dfs.append(pd.read_parquet(os.path.join(ROOT_TRAIN, path)).groupby('id').mean())


# new test_data
for key, path in enumerate(test_paths):
    test_dfs.append(pd.read_parquet(os.path.join(ROOT_TEST, path)).groupby('id').mean())

In [4]:
train_df = pd.concat((train_dfs))
test_df = pd.concat((test_dfs))

In [5]:
train_target = pd.read_csv('data/train_target.csv')
test_target = pd.read_csv('data/test_target.csv')

In [6]:
train_df = pd.merge(train_df, train_target, how='left', on='id')
train_df.to_csv('train_df_mean.csv', index=False)
test_df.to_csv('test_df_mean.csv')

In [7]:
train_df.to_parquet('train_df_mean.pq', index=False)
test_df.to_parquet('test_df_mean.pq')

In [13]:
X = train_df.drop(['id', 'flag'], axis=1)
y = train_df['flag']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [16]:
cat = CatBoostClassifier(iterations=3_000,
                         depth=5,
                         learning_rate=0.05,
                         task_type='gpu')

cat.fit(X_train, y_train, eval_set=(X_test, y_test))

0:	learn: 0.6177636	test: 0.6177152	best: 0.6177152 (0)	total: 782ms	remaining: 39m 4s
1:	learn: 0.5530520	test: 0.5532402	best: 0.5532402 (1)	total: 1.44s	remaining: 36m 2s
2:	learn: 0.4983497	test: 0.4985422	best: 0.4985422 (2)	total: 1.98s	remaining: 33m 1s
3:	learn: 0.4507641	test: 0.4508375	best: 0.4508375 (3)	total: 2.46s	remaining: 30m 39s
4:	learn: 0.4097489	test: 0.4098518	best: 0.4098518 (4)	total: 2.93s	remaining: 29m 17s
5:	learn: 0.3747515	test: 0.3748767	best: 0.3748767 (5)	total: 3.48s	remaining: 28m 56s
6:	learn: 0.3445712	test: 0.3447210	best: 0.3447210 (6)	total: 4.05s	remaining: 28m 51s
7:	learn: 0.3181636	test: 0.3183396	best: 0.3183396 (7)	total: 4.8s	remaining: 29m 57s
8:	learn: 0.2956023	test: 0.2958045	best: 0.2958045 (8)	total: 5.35s	remaining: 29m 38s
9:	learn: 0.2762882	test: 0.2765131	best: 0.2765131 (9)	total: 6.02s	remaining: 29m 59s
10:	learn: 0.2594675	test: 0.2597068	best: 0.2597068 (10)	total: 6.72s	remaining: 30m 27s
11:	learn: 0.2452859	test: 0.24554

KeyboardInterrupt: 